# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 08:07:57.524000 1454191 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:07:57.524000 1454191 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:07:58] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36028, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=228045724, constrained_

[2025-08-26 08:07:58] Using default HuggingFace chat template with detected content format: string


W0826 08:08:06.153000 1454406 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:08:06.153000 1454406 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0826 08:08:06.624000 1454407 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:08:06.624000 1454407 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=53.65 GB): 100%|██████████| 4/4 [00:04<00:00,  1.22s/it]
[2025-08-26 08:08:21] Capture draft cuda graph end. Time elapsed: 5.34 s. mem usage=0.14 GB. avail mem=53.64 GB.
[2025-08-26 08:08:21] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.64 GB


Capturing batches (bs=1 avail_mem=53.56 GB): 100%|██████████| 4/4 [00:00<00:00, 110.36it/s]
[2025-08-26 08:08:22] Capture draft extend cuda graph end. Time elapsed: 0.94 s. mem usage=0.08 GB. avail mem=53.56 GB.
[2025-08-26 08:08:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.56 GB
[2025-08-26 08:08:23] INFO:     Started server process [1454191]
[2025-08-26 08:08:23] INFO:     Waiting for application startup.


[2025-08-26 08:08:23] INFO:     Application startup complete.
[2025-08-26 08:08:23] INFO:     Uvicorn running on http://127.0.0.1:36028 (Press CTRL+C to quit)


[2025-08-26 08:08:23] INFO:     127.0.0.1:36980 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 08:08:24] INFO:     127.0.0.1:36984 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 08:08:24] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:08:24] INFO:     127.0.0.1:36992 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 08:08:24] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 08:08:28] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:08:28] INFO:     127.0.0.1:36866 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 08:08:36.445000 1455213 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:08:36.445000 1455213 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:08:37] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34119, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=331657444, constrained_js

[2025-08-26 08:08:37] Using default HuggingFace chat template with detected content format: string


W0826 08:08:44.325000 1455427 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:08:44.325000 1455427 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0826 08:08:44.575000 1455428 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:08:44.575000 1455428 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=53.49 GB): 100%|██████████| 4/4 [00:07<00:00,  1.80s/it]
[2025-08-26 08:09:10] Capture draft cuda graph end. Time elapsed: 7.91 s. mem usage=0.23 GB. avail mem=53.45 GB.
[2025-08-26 08:09:10] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.45 GB


Capturing batches (bs=1 avail_mem=53.35 GB): 100%|██████████| 4/4 [00:00<00:00, 37.38it/s]
[2025-08-26 08:09:11] Capture draft extend cuda graph end. Time elapsed: 0.89 s. mem usage=0.10 GB. avail mem=53.35 GB.
[2025-08-26 08:09:11] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.35 GB


[2025-08-26 08:09:11] INFO:     Started server process [1455213]
[2025-08-26 08:09:11] INFO:     Waiting for application startup.
[2025-08-26 08:09:11] INFO:     Application startup complete.
[2025-08-26 08:09:11] INFO:     Uvicorn running on http://127.0.0.1:34119 (Press CTRL+C to quit)


[2025-08-26 08:09:12] INFO:     127.0.0.1:42168 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 08:09:12] INFO:     127.0.0.1:42180 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 08:09:12] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 08:09:12] INFO:     127.0.0.1:42190 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 08:09:12] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 08:09:17] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:09:17] INFO:     127.0.0.1:42880 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 08:09:25.136000 1456500 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:09:25.136000 1456500 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:09:26] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30450, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=990936672,

[2025-08-26 08:09:27] Using default HuggingFace chat template with detected content format: string


W0826 08:09:34.643000 1456716 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:09:34.643000 1456716 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 08:09:34.647000 1456715 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:09:34.647000 1456715 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:09:35] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.39 GB): 100%|██████████| 4/4 [00:06<00:00,  1.59s/it]
[2025-08-26 08:10:02] Capture draft cuda graph end. Time elapsed: 6.86 s. mem usage=0.22 GB. avail mem=58.34 GB.
[2025-08-26 08:10:02] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.34 GB


Capturing batches (bs=1 avail_mem=58.25 GB): 100%|██████████| 4/4 [00:00<00:00, 23.98it/s]
[2025-08-26 08:10:03] Capture draft extend cuda graph end. Time elapsed: 0.73 s. mem usage=0.10 GB. avail mem=58.25 GB.
[2025-08-26 08:10:03] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=58.25 GB


[2025-08-26 08:10:03] INFO:     Started server process [1456500]
[2025-08-26 08:10:03] INFO:     Waiting for application startup.
[2025-08-26 08:10:03] INFO:     Application startup complete.
[2025-08-26 08:10:03] INFO:     Uvicorn running on http://127.0.0.1:30450 (Press CTRL+C to quit)


[2025-08-26 08:10:04] INFO:     127.0.0.1:32960 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 08:10:04] INFO:     127.0.0.1:32970 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 08:10:04] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 08:10:04] INFO:     127.0.0.1:32980 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 08:10:04] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 08:10:09] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:10:09] INFO:     127.0.0.1:32986 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 08:10:16.827000 1457561 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:10:16.827000 1457561 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:10:17] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36255, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=865771739, const

[2025-08-26 08:10:18] Using default HuggingFace chat template with detected content format: string


W0826 08:10:25.387000 1457778 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:10:25.387000 1457778 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 08:10:25.478000 1457777 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:10:25.478000 1457777 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:10:25] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.24 GB):  75%|███████▌  | 3/4 [00:02<00:00,  1.38it/s]

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:04<00:00,  1.21s/it]
[2025-08-26 08:10:51] Capture draft cuda graph end. Time elapsed: 5.33 s. mem usage=0.23 GB. avail mem=58.19 GB.
[2025-08-26 08:10:51] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.19 GB


Capturing batches (bs=1 avail_mem=58.08 GB): 100%|██████████| 4/4 [00:00<00:00, 84.81it/s]
[2025-08-26 08:10:52] Capture draft extend cuda graph end. Time elapsed: 0.81 s. mem usage=0.11 GB. avail mem=58.08 GB.
[2025-08-26 08:10:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.08 GB


[2025-08-26 08:10:52] INFO:     Started server process [1457561]
[2025-08-26 08:10:52] INFO:     Waiting for application startup.
[2025-08-26 08:10:52] INFO:     Application startup complete.
[2025-08-26 08:10:52] INFO:     Uvicorn running on http://127.0.0.1:36255 (Press CTRL+C to quit)
[2025-08-26 08:10:52] INFO:     127.0.0.1:58826 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 08:10:53] INFO:     127.0.0.1:58832 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 08:10:53] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:10:54] INFO:     127.0.0.1:58846 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 08:10:54] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 08:10:57] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 08:10:58] INFO:     127.0.0.1:34834 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 08:11:05.425000 1458583 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:11:05.425000 1458583 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:11:06] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32242, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=997692844, constrained_json_whitespace_patte

[2025-08-26 08:11:07] Using default HuggingFace chat template with detected content format: string


W0826 08:11:14.849000 1458798 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:11:14.849000 1458798 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 08:11:14.966000 1458799 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 08:11:14.966000 1458799 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 08:11:15] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-26 08:11:15] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=59.45 GB): 100%|██████████| 4/4 [00:02<00:00,  1.97it/s]
[2025-08-26 08:11:27] Capture draft cuda graph end. Time elapsed: 2.62 s. mem usage=0.01 GB. avail mem=59.45 GB.
[2025-08-26 08:11:27] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=59.45 GB


Capturing batches (bs=3 avail_mem=59.37 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=59.35 GB): 100%|██████████| 4/4 [00:00<00:00, 14.53it/s]
[2025-08-26 08:11:28] Capture draft extend cuda graph end. Time elapsed: 1.23 s. mem usage=0.09 GB. avail mem=59.35 GB.
[2025-08-26 08:11:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=59.35 GB


[2025-08-26 08:11:29] INFO:     Started server process [1458583]
[2025-08-26 08:11:29] INFO:     Waiting for application startup.
[2025-08-26 08:11:29] INFO:     Application startup complete.
[2025-08-26 08:11:29] INFO:     Uvicorn running on http://0.0.0.0:32242 (Press CTRL+C to quit)
[2025-08-26 08:11:29] INFO:     127.0.0.1:44136 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 08:11:30] INFO:     127.0.0.1:44150 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 08:11:30] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 08:11:30] INFO:     127.0.0.1:44152 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 08:11:30] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-26 08:11:34] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 08:11:34] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, accept len: 1.64, cuda graph: True, gen throughput (token/s): 11.16, #queue-req: 0, 


[2025-08-26 08:11:35] Decode batch. #running-req: 1, #token: 154, token usage: 0.01, accept len: 1.70, cuda graph: True, gen throughput (token/s): 170.08, #queue-req: 0, 


[2025-08-26 08:11:35] Decode batch. #running-req: 1, #token: 219, token usage: 0.01, accept len: 1.62, cuda graph: True, gen throughput (token/s): 156.84, #queue-req: 0, 


[2025-08-26 08:11:35] Decode batch. #running-req: 1, #token: 283, token usage: 0.01, accept len: 1.60, cuda graph: True, gen throughput (token/s): 159.64, #queue-req: 0, 


[2025-08-26 08:11:36] Decode batch. #running-req: 1, #token: 359, token usage: 0.02, accept len: 1.90, cuda graph: True, gen throughput (token/s): 190.11, #queue-req: 0, 


[2025-08-26 08:11:36] Decode batch. #running-req: 1, #token: 424, token usage: 0.02, accept len: 1.62, cuda graph: True, gen throughput (token/s): 162.39, #queue-req: 0, 
[2025-08-26 08:11:36] INFO:     127.0.0.1:44168 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).